###  installing and importing packages

In [6]:
pip install betterreads

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
from betterreads import client

###  Data collection approach

I am starting with a Kaggle dataset from  https://www.kaggle.com/zygmunt/goodbooks-10k

# TODO < include > eda


additional data collection for 
*Book description 
*number of pages
*is_book_enum

In [9]:
#  Get the API keys here https://www.goodreads.com/api/keys

gc = client.GoodreadsClient('zIHHGGHATvYay6dTTZ1AqA',
                            'xFiCr7PBabSezd7zpXhqNyvsIDFfoX4ftC6Pww7v2g')

In [10]:
books = pd.read_csv('../data/books.csv')
print( f"shape of the books dataframe is {books.shape}")
books.head(1)

shape of the books dataframe is (10000, 26)


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,description,num_pages,is_ebook
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,"Could you survive on your own, in the wild, wi...",374.0,False


In [12]:
books['description'].isna().sum()
missing_desc_books = books[books['description'].isna()]['book_id'].values
missing_desc_books

array([ 728,  977,  992,  994, 1108, 1113, 1329, 1338, 1382, 1432, 1475,
       1666, 1679, 1691, 1754, 1935, 2068, 2100, 2139, 2151, 2397, 2680,
       2685, 3054, 3413, 3438, 3484, 3577, 3636, 3813, 4154, 4157, 4264,
       4291, 4309, 4330, 4504, 4601, 4676, 4717, 4835, 4980, 5002, 5027,
       5094, 5376, 5540, 5706, 5802, 5898, 5914, 5957, 6022, 6141, 6240,
       6251, 6372, 6457, 6514, 6518, 6524, 6530, 6561, 6586, 6778, 6938,
       6959, 7114, 7196, 7239, 7263, 7428, 7484, 7767, 7795, 7803, 7813,
       7838, 7990, 8011, 8033, 8038, 8197, 8214, 8336, 8372, 8390, 8441,
       8472, 8665, 8696, 8755, 8761, 8854, 8880, 8903, 8947, 8971, 9076,
       9120, 9137, 9183, 9220, 9241, 9252, 9327, 9343, 9555, 9667, 9714,
       9816, 9818, 9900, 9974, 9990])

In [13]:
#https://betterreads.readthedocs.io/en/latest/book.html
#for i in range(1,4000):
for i in missing_desc_books:
    goodreads_book_id = books.loc[books['book_id'] == i,'goodreads_book_id'].values[0]
    try:
        book_obj= get_book(goodreads_book_id)
        print( f"{i},{book_obj}")
        books.loc[books['book_id'] == i,'description']= book_obj.description
        try:
            books.loc[books['book_id'] == i,'num_pages']=  book_obj.num_pages
        except:
            books.loc[books['book_id'] == i,'num_pages']=  0

        books.loc[books['book_id'] == i,'is_ebook']=book_obj.is_ebook  
    except:
        print(f"failed to parse {i}")

failed to parse 728
failed to parse 977
failed to parse 992
failed to parse 994
failed to parse 1108
failed to parse 1113
failed to parse 1329
failed to parse 1338
failed to parse 1382
failed to parse 1432
failed to parse 1475
failed to parse 1666
failed to parse 1679
failed to parse 1691
failed to parse 1754
failed to parse 1935
failed to parse 2068
failed to parse 2100
failed to parse 2139
failed to parse 2151
failed to parse 2397
failed to parse 2680
failed to parse 2685
failed to parse 3054
failed to parse 3413
failed to parse 3438
failed to parse 3484
failed to parse 3577
failed to parse 3636
failed to parse 3813
failed to parse 4154
failed to parse 4157
failed to parse 4264
failed to parse 4291
failed to parse 4309
failed to parse 4330
failed to parse 4504
failed to parse 4601
failed to parse 4676
failed to parse 4717
failed to parse 4835
failed to parse 4980
failed to parse 5002
failed to parse 5027
failed to parse 5094
failed to parse 5376
failed to parse 5540
failed to parse 5

In [15]:
# Read book ratings
ratings = pd.read_csv("../data/ratings.csv")
print( f"shape of the books dataframe is {ratings.shape}")
ratings.head(1)

shape of the books dataframe is (5976479, 3)


,user_id,book_id,rating
0,1,258,5


In [16]:
books.rename({'original_title':'title'},inplace=True)
#books =pd.merge(books,ratings, how ='inner',left_on ='book_id',right_on='book_id')
#books.head(1)

In [17]:
book_tags = pd.read_csv("../data/book_tags.csv")
print( f"shape of the books dataframe is {book_tags.shape}")
book_tags.head(1)

shape of the books dataframe is (999912, 3)


,goodreads_book_id,tag_id,count
0,1,30574,167697


In [18]:
tags = pd.read_csv("../data/tags.csv")
print( f"shape of the books dataframe is {tags.shape}")
tags.head(1)

shape of the books dataframe is (34252, 2)


,tag_id,tag_name
0,0,-


In [19]:
book_tags =pd.merge(left =book_tags,right=tags, how="left")
book_tags.head(1)

,goodreads_book_id,tag_id,count,tag_name
0,1,30574,167697,to-read


In [20]:
#books =pd.merge(books,book_tags, how ='inner',left_on ='goodreads_book_id',right_on='goodreads_book_id')
#books.head(1)

In [21]:
def get_book(id):
    book = gc.book(id)
    return book

In [22]:
books.to_csv("../data/books.csv", index=False)


In [ ]:
# Combine all data frames in to one

######  references
https://pypi.org/project/Goodreads/

https://betterreads.readthedocs.io/en/latest/

https://www.kaggle.com/zygmunt/goodbooks-10k